In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# import altair as alt

pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [3]:
df = pd.read_csv('SBAnational.csv')
df.head(1)

# Columns to drop
# 1. LoanNr_ChkDgt
# 2. Name
# 3. Drop city + zip - focus state to get a general idea and reasonable trend

# Columns to modify or add
# 1. ApprovalDate - convert to datetime (either by Month + year or just Year)
# 2. NAICS - concat the industry for each NAICS 

# Unsure exactly how to handle
# 1. Create job (Does it play a role in the loan approval?)
# 2. Retained Job (Does it play a role in the loan approval?)
# 3. FranchiseCode (Remake to boolean or actually finding franchise of corresponding code?)

/var/folders/d3/6_5r7d716cb8gqs9y4qft_600000gp/T/ipykernel_56820/2833908245.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('SBAnational.csv')


,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,28-Feb-97,1997,84,4,2.0,0,0,1,0,N,Y,NaN,28-Feb-99,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$48,000.00"


In [4]:
import geopandas
state = geopandas.read_file('ne_110m_admin_1_states_provinces')

# manipulate string data to match the loan dataset, so instead of US-MN, it will be MN, for Minnesota etc
state['iso_3166_2'] = state['iso_3166_2'].str.replace('US-', '')

state.head(1)

,featurecla,scalerank,adm1_code,diss_me,iso_3166_2,wikipedia,iso_a2,adm0_sr,name,name_alt,name_local,type,type_en,code_local,code_hasc,note,hasc_maybe,region,region_cod,provnum_ne,gadm_level,check_me,datarank,abbrev,postal,area_sqkm,sameascity,labelrank,name_len,mapcolor9,mapcolor13,fips,fips_alt,woe_id,woe_label,woe_name,latitude,longitude,sov_a3,adm0_a3,adm0_label,admin,geonunit,gu_a3,gn_id,gn_name,gns_id,gns_name,gn_level,gn_region,gn_a1_code,region_sub,sub_code,gns_level,gns_lang,gns_adm1,gns_region,min_label,max_label,min_zoom,wikidataid,name_ar,name_bn,name_de,name_en,name_es,name_fr,name_el,name_hi,name_hu,name_id,name_it,name_ja,name_ko,name_nl,name_pl,name_pt,name_ru,name_sv,name_tr,name_vi,name_zh,ne_id,name_he,name_uk,name_ur,name_fa,name_zht,FCLASS_ISO,FCLASS_US,FCLASS_FR,FCLASS_RU,FCLASS_ES,FCLASS_CN,FCLASS_TW,FCLASS_IN,FCLASS_NP,FCLASS_PK,FCLASS_DE,FCLASS_GB,FCLASS_BR,FCLASS_IL,FCLASS_PS,FCLASS_SA,FCLASS_EG,FCLASS_MA,FCLASS_PT,FCLASS_AR,FCLASS_JP,FCLASS_KO,FCLASS_VN,FCLASS_TR,FCLASS_ID,FCLASS_PL,FCLASS_GR,FCLASS_IT,FCLASS_NL,FCLASS_SE,FCLASS_BD,FCLASS_UA,FCLASS_TLC,geometry
0,Admin-1 scale rank,2,USA-3514,3514,MN,http://en.wikipedia.org/wiki/Minnesota,US,1,Minnesota,MN|Minn.,None,State,State,US27,US.MN,None,None,Midwest,None,0,1,20,1,Minn.,MN,0,-99,0,9,1,1,US27,None,2347582,"Minnesota, US, United States",Minnesota,46.0592,-93.364,US1,USA,2,United States of America,United States of America,USA,5037779,Minnesota,-1,None,1,None,US.MN,West North Central,None,-1,None,None,None,3.5,7.5,2,Q1527,مينيسوتا,মিনেসোটা,Minnesota,Minnesota,Minnesota,Minnesota,Μινεσότα,मिनेसोटा,Minnesota,Minnesota,Minnesota,ミネソタ州,미네소타,Minnesota,Minnesota,Minnesota,Миннесота,Minnesota,Minnesota,Minnesota,明尼苏达州,1159315297,מינסוטה,Міннесота,مینیسوٹا,مینهسوتا,明尼蘇達州,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"POLYGON ((-89.95766 47.28691, -90.13175 47.292..."


In [5]:
merged_init = pd.merge(df, state, left_on='State', right_on='iso_3166_2')

# selecting all columns up to and including 'region'
region_col_index = merged_init.columns.get_loc('region') + 1  # +1 to include 'region' column itself
merged_up_to_region = merged_init.iloc[:, :region_col_index]

# selecting the geometry column
merged_geometry = merged_init[['geometry']]

# concatenating the parts to get the final DataFrame
merged_final = pd.concat([merged_up_to_region, merged_geometry], axis=1)

# dropping final rows that are not needed now that data is aggregated
merged_final = merged_final.drop(columns=['hasc_maybe', 'code_hasc', 'type_en', 'name_local', 'name_alt', 'adm0_sr', 'iso_a2', 
                                          'wikipedia', 'diss_me', 'scalerank', 'featurecla', 'note', 'code_local'])

merged_final.head(1)

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv,adm1_code,iso_3166_2,name,type,region,geometry
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,28-Feb-97,1997,84,4,2.0,0,0,1,0,N,Y,NaN,28-Feb-99,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$48,000.00",USA-3547,IN,Indiana,State,Midwest,"POLYGON ((-88.05108 37.81960, -88.01881 38.021..."


In [6]:
# making industry column with first two digits of NAICS
merged_industry = merged_final.copy()
merged_industry['NAICS'] = merged_industry['NAICS'].astype(str)
merged_industry['Industry'] = merged_industry['NAICS'].str[:2]


merged_industry.head(1)

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv,adm1_code,iso_3166_2,name,type,region,geometry,Industry
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,28-Feb-97,1997,84,4,2.0,0,0,1,0,N,Y,NaN,28-Feb-99,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$48,000.00",USA-3547,IN,Indiana,State,Midwest,"POLYGON ((-88.05108 37.81960, -88.01881 38.021...",45


In [7]:
# moving column to next to NAICS
columns = list(merged_industry.columns)

columns.remove('Industry')

target_index = columns.index('NAICS') + 1  
columns.insert(target_index, 'Industry')

merged_industry = merged_industry[columns]

merged_industry.head(1)

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,Industry,ApprovalDate,ApprovalFY,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv,adm1_code,iso_3166_2,name,type,region,geometry
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,45,28-Feb-97,1997,84,4,2.0,0,0,1,0,N,Y,NaN,28-Feb-99,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$48,000.00",USA-3547,IN,Indiana,State,Midwest,"POLYGON ((-88.05108 37.81960, -88.01881 38.021..."


In [8]:
# map NAISC to industry

industry_map = {
  '11': 'Ag/For/Fish/Hunt',
  '21': 'Min/Quar/Oil_Gas_ext',
  '22': 'Utilities',
  '23': 'Construction',
  '31': 'Manufacturing',
  '32': 'Manufacturing',
  '33': 'Manufacturing',
  '42': 'Wholesale_trade',
  '44': 'Retail_trade',
  '45': 'Retail_trade',
  '48': 'Trans/Ware',
  '49': 'Trans/Ware',
  '51': 'Information',
  '52': 'Finance/Insurance',
  '53': 'RE/Rental/Lease',
  '54': 'Prof/Science/Tech',
  '55': 'Mgmt_comp',
  '56': 'Admin_sup/Waste_Mgmt_Rem',
  '61': 'Educational',
  '62': 'Healthcare/Social_assist',
  '71': 'Arts/Entertain/Rec',
  '72': 'Accom/Food_serv',
  '81': 'Other_no_pub',
  '92': 'Public_Admin'
}

merged_industry['Industry'] = merged_industry['Industry'].map(industry_map)


In [15]:
# making sure industry works
merged_industry.head()

# export_csv_df = merged_industry.to_csv('exported.csv', index=False)

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,Industry,ApprovalDate,ApprovalFY,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv,adm1_code,iso_3166_2,name,type,region,geometry
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,Retail_trade,28-Feb-97,1997,84,4,2.0,0,0,1,0,N,Y,NaN,28-Feb-99,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$48,000.00",USA-3547,IN,Indiana,State,Midwest,"POLYGON ((-88.05108 37.81960, -88.01881 38.021..."
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,Accom/Food_serv,28-Feb-97,1997,60,2,2.0,0,0,1,0,N,Y,NaN,31-May-97,"$40,000.00",$0.00,P I F,$0.00,"$40,000.00","$32,000.00",USA-3547,IN,Indiana,State,Midwest,"POLYGON ((-88.05108 37.81960, -88.01881 38.021..."
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,Healthcare/Social_assist,28-Feb-97,1997,180,7,1.0,0,0,1,0,N,N,NaN,31-Dec-97,"$287,000.00",$0.00,P I F,$0.00,"$287,000.00","$215,250.00",USA-3547,IN,Indiana,State,Midwest,"POLYGON ((-88.05108 37.81960, -88.01881 38.021..."
3,1000653000,LARRY SCHOETTMER FORD INC,EDINBURGH,IN,46124,JPMORGAN CHASE BANK NATL ASSOC,IN,0,NaN,11-Jun-80,1980,120,16,2.0,0,0,0,0,Y,N,4-Oct-89,31-Jul-80,"$197,485.00",$0.00,CHGOFF,"$44,374.00","$200,000.00","$150,000.00",USA-3547,IN,Indiana,State,Midwest,"POLYGON ((-88.05108 37.81960, -88.01881 38.021..."
4,1001034000,CD WAREHOUSE,GREENWOOD,IN,46142,"PNC BANK, NATIONAL ASSOCIATION",IN,451220,Retail_trade,28-Feb-97,1997,66,2,2.0,0,0,10656,0,N,Y,NaN,31-Aug-97,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$48,000.00",USA-3547,IN,Indiana,State,Midwest,"POLYGON ((-88.05108 37.81960, -88.01881 38.021..."
